## Leren: Programming Assignment 2
#### Wim Berkelmans, 10793674
#### Philip Bouman, 10668667


### 1. Read data from file

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as linalg

def loadData(feature):
    # load input:
    # features: Bedrooms, Bathrooms, Size
    # target: Price
    data = np.loadtxt('HousesRegr.csv', skiprows = 1, delimiter=';');
    columns = ['MLS','Bedrooms','Bathrooms','Size','Price']
    index = columns.index(feature)
    return data[:,index]

bedroom = loadData('Bedrooms')
bathroom = loadData('Bathrooms')
size = loadData('Size')
price = loadData('Price')

# create easy format and add x_0's
def designMatrix(n):
    data = np.loadtxt('HousesRegr.csv', skiprows = 1, delimiter=';');
    m = len(data)
    X = np.ones((m,n))
    for i in range(m):
        for j in range(1,n):
            X[i, j] = data[i,j]
    return X

# vectorize predicted output
def targetVec(y):
    Y = y[:, np.newaxis] 
    return Y

In [12]:
def regressionVec(X, Y):
    
    # apply normal equation
    norm = np.dot(np.dot(linalg.inv(np.dot(X.T, X)), X.T), Y)
    
    return norm

X = np.array([[1,2,3],[1,4,5],[1,4,3]])
Y = np.array([6,6,10])

# X = designMatrix(4)
# Y = targetVec(price)

print X
# print Y

Theta = regressionVec(X, Y)
print Theta

[[1 2 3]
 [1 4 5]
 [1 4 3]]
[ 8.  2. -2.]


In [14]:
def calcCostVec(X, Y, Theta):
    m = len(Y)
    errorSquared = (np.dot(X, Theta) - Y)**2
    
    # sum over errorSquared by multiplying with a rowvector of ones
    ones = np.ones(m)
    cost = np.dot(ones.T, errorSquared)/(2.0*m)
    return cost

cost = calcCostVec(X, Y, Theta)
print cost

5.37740183726e-29


In [46]:
def gradientIter(thetas, y, *args):
    m = len(args[0])
    n = len(args)
    sum_thetas = np.zeros(n+1)
    print sum_thetas
    print m
    print n
    
#     for i in range(m):
#         sum_theta_0 += theta_ * x[i] + theta_0 - y[i]
        
#         sum_theta_1 += x[i] * (theta_1 * x[i] + theta_0 - y[i])
        
#         sum_theta_2 += x[i] * (theta_2 + x[i] + theta_1 * x[i] + theta_0 - y[i])
        
    for i in range(m):
        for j in range(n+1):
            h = 0
            for k in range(n-1):
                h += thetas[0]
                h += thetas[k+1] * args[k+1][i]
                print h    
        
        sum_thetas[j] += h - y[j]    
                
                
        

#     new_theta_0 = theta_0 - ((1.0/m)*sum_theta_0 * alpha)
#     new_theta_1 =theta_1 - ((1.0/m)*sum_theta_1 * alpha)

thetas = [0.5,0.5]
print bedroom

gradientIter(price, thetas, bedroom)

[  3.   4.   4.   4.   3.   3.   4.   3.   4.   3.   3.   3.   2.   3.   3.
   4.   4.   3.   3.   3.   4.   3.   2.   3.   3.   4.   3.   4.   3.   3.
   3.   3.   3.   3.   3.   2.   2.   3.   4.   3.   4.   3.   3.   3.   3.
   4.   3.   3.   4.   4.   3.   3.   2.   4.   3.   2.   3.   4.   4.   4.
   3.   3.   3.   3.   3.   2.   3.   3.   2.   4.   4.   3.   3.   4.   3.
   7.   1.   4.   3.   3.   4.   4.   1.   3.   3.   2.   3.   4.   3.   3.
   3.   3.   3.   3.   3.   5.   3.   3.   3.   3.   3.   3.   0.   4.   3.
   3.   2.   3.   2.   4.   4.   4.   4.   3.   2.   4.   4.   3.   3.   3.
   3.   3.   5.   4.   4.   4.   4.   2.   2.   3.   4.   1.   2.   2.   3.
   3.   4.   2.   2.   2.   3.   3.   3.   3.   3.   3.   2.   3.   3.   3.
   4.   4.   3.   2.   3.   3.   4.   3.   3.   4.   5.   3.   3.   2.   2.
   4.   3.   3.   4.   2.   2.   3.   5.   1.   3.   5.   2.   3.   3.   3.
   3.   3.   3.   3.   2.   3.   3.   5.   2.   2.   2.   2.   2.   2.   4.
   3.   4.  

In [25]:
def regressionIter(*args):
    print args[0]
#     for a in args:
#         print a
        
regressionIter(bedroom, bathroom, price)        


[  3.   4.   4.   4.   3.   3.   4.   3.   4.   3.   3.   3.   2.   3.   3.
   4.   4.   3.   3.   3.   4.   3.   2.   3.   3.   4.   3.   4.   3.   3.
   3.   3.   3.   3.   3.   2.   2.   3.   4.   3.   4.   3.   3.   3.   3.
   4.   3.   3.   4.   4.   3.   3.   2.   4.   3.   2.   3.   4.   4.   4.
   3.   3.   3.   3.   3.   2.   3.   3.   2.   4.   4.   3.   3.   4.   3.
   7.   1.   4.   3.   3.   4.   4.   1.   3.   3.   2.   3.   4.   3.   3.
   3.   3.   3.   3.   3.   5.   3.   3.   3.   3.   3.   3.   0.   4.   3.
   3.   2.   3.   2.   4.   4.   4.   4.   3.   2.   4.   4.   3.   3.   3.
   3.   3.   5.   4.   4.   4.   4.   2.   2.   3.   4.   1.   2.   2.   3.
   3.   4.   2.   2.   2.   3.   3.   3.   3.   3.   3.   2.   3.   3.   3.
   4.   4.   3.   2.   3.   3.   4.   3.   3.   4.   5.   3.   3.   2.   2.
   4.   3.   3.   4.   2.   2.   3.   5.   1.   3.   5.   2.   3.   3.   3.
   3.   3.   3.   3.   2.   3.   3.   5.   2.   2.   2.   2.   2.   2.   4.
   3.   4.  